@https://angle.readthedocs.io/en/latest/notes/quickstart.html
@article{li2023angle,
  title={AnglE-Optimized Text Embeddings},
  author={Li, Xianming and Li, Jing},
  journal={arXiv preprint arXiv:2309.12871},
  year={2023}
}


In [ ]:
!python -m pip install -U angle-emb billm

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import pandas as pd
from angle_emb import AnglE
from angle_emb.utils import cosine_similarity
import torch
from torch.utils.data import DataLoader
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#Nous faisons juste un test pour voir si tout est bien installé


angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()
# for non-retrieval tasks, we don't need to specify prompt when using UAE-Large-V1.
doc_vecs = angle.encode([
    'AI developers and researchers focusing on anomaly detection algorithms for non-intrusive inspection systems.',
    'Smugglers'
])

for i, dv1 in enumerate(doc_vecs):
    for dv2 in doc_vecs[i+1:]:
        print(cosine_similarity(dv1, dv2))

0.4376459491586401


In [ ]:

###################################### llama-3/human

# Lire les données depuis le fichier Excel
df = pd.read_excel('/2023 fichier de base Copie.xlsx')
df.iloc[:, 12] = df.iloc[:, 12].fillna('neutral')
# Assurez-vous que les colonnes 10 et 11 existent et contiennent les textes
col10_texts = df.iloc[:, 11].dropna().tolist()  # Target population using Llama-3
col11_texts = df.iloc[:, 12].dropna().tolist()  # Target population using llm and human review


# Vérifiez que les deux listes ont la même longueur
if len(col10_texts) != len(col11_texts):
    raise ValueError("Les colonnes 10 et 11 doivent avoir le même nombre de textes.")

# Instancier le modèle AnglE
angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()

# Encoder les textes des deux colonnes
doc_vecs_col10 = angle.encode(col10_texts)
doc_vecs_col11 = angle.encode(col11_texts)

similarities = [cosine_similarity(dv1, dv2) for dv1, dv2 in zip(doc_vecs_col10, doc_vecs_col11)]

# Ajouter les résultats au DataFrame dans la colonne 15 (index 14)
df.insert(13, 'Similarité Cosinus llama-3/human m1', pd.Series(similarities))

df.to_excel('/2023 fichier de base Copie.xlsx', index=False)

OutOfMemoryError: CUDA out of memory. Tried to allocate 118.00 MiB. GPU 

In [ ]:
###################################### Qwen-2/human


# Lire les données depuis le fichier Excel
df = pd.read_excel('/2023 fichier de base Copie.xlsx')
df.iloc[:, 12] = df.iloc[:, 12].fillna('neutral')

# Assurez-vous que les colonnes 10 et 11 existent et contiennent les textes
col10_texts = df.iloc[:, 10].dropna().tolist()  # Target population using Qwen-2
col11_texts = df.iloc[:, 12].dropna().tolist()  # Target population using llm and human review

# Vérifiez que les deux listes ont la même longueur
if len(col10_texts) != len(col11_texts):
    raise ValueError("Les colonnes 10 et 11 doivent avoir le même nombre de textes.")

# Instancier le modèle AnglE
angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()

# Définir une taille de batch en fonction de la mémoire disponible
batch_size = 64  # Essayez d'augmenter ou de réduire cette valeur selon la mémoire disponible

# Fonction pour encoder en batch
def encode_in_batches(texts, model, batch_size):
    dataloader = DataLoader(texts, batch_size=batch_size, shuffle=False)
    all_encodings = []
    for batch in dataloader:
        encodings = model.encode(batch)
        all_encodings.extend(encodings)
        torch.cuda.empty_cache()  # Libérer la mémoire GPU après chaque batch
    return all_encodings

# Encoder les textes des deux colonnes en batches
doc_vecs_col10 = encode_in_batches(col10_texts, angle, batch_size)
doc_vecs_col11 = encode_in_batches(col11_texts, angle, batch_size)

# Calculer les similarités cosinus
similarities = [cosine_similarity([dv1], [dv2])[0][0] for dv1, dv2 in zip(doc_vecs_col10, doc_vecs_col11)]

# Ajouter les résultats au DataFrame dans la colonne 15 (index 14)
df.insert(14, 'Similarité Cosinus Qwen2/human m1', pd.Series(similarities))

# Sauvegarder les résultats dans le fichier Excel
df.to_excel('/2023 fichier de base Copie.xlsx', index=False)
